In [1]:
import csv
import numpy as np

import collections
import operator
from functools import reduce
import json
from multiprocessing import Pool

In [2]:
import xmnlp
xmnlp.radical('花丸晴琉')

(Lazy Load) Loading model...


['艹', '丿', '日', '王']

In [3]:
# Build up Chinese sub-character unites diactionary.

chaiZi_Dic = {}
with open("./corpus/chaizi/chaizi-jt.txt", 'r') as f:
    reader = csv.reader(f,delimiter='\t')
    for row in reader:
        chaiZi_Dic[row[0]] = row[1]
        
def getSubChar(charCN):
    if(charCN in chaiZi_Dic.keys()):
        return chaiZi_Dic[item]
    else:
        return [None]

In [4]:
print(len(chaiZi_Dic.keys()))
for item in "花丸A晴琉":
    print(getSubChar(item))
    print(xmnlp.radical(item))
    print()

14563
草 化
['艹']

丶 九
['丿']

[None]
[None]

日 青
['日']

玉 亠 厶 川
['王']



In [5]:
## Create Corpus for Chinese Word Segamentation

In [6]:
import pandas as pd
pkuTrain = "./corpus/cws/icwb2-data/training/pku_training.utf8"
train = pd.read_table(pkuTrain,  encoding='utf8', header=None, names=['input'])  # don't drop the empty lines yet, they show up as NaN in the data frame
train.head(n=1)


,input
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...


In [7]:
def prepross_data(train_Data):
    inputs = train_Data["input"][:]
    print(len(inputs))
    raws = []
    tokenList = []
    
    for item in inputs:
        item = item.replace("   ", "  ")

        raw = item.replace(" ", "")
#         print(item)
#         print(raw)
        raws.append(raw)
        tokens = (item.split("  ")[:-1])
        tokenList.append(tokens)
#         print(tokens)
    train_Data["raws"] = raws
    train_Data["tokenList"] = tokenList
    dictionary_train_word = list(set(reduce(operator.add,train_Data["tokenList"])))
    dictionary_train_char =  list(set(reduce(operator.add,train_Data["raws"])))    
    return train_Data, dictionary_train_char, dictionary_train_word
train, train_dic_char, train_dic_word  = prepross_data(train)

19054


In [8]:
# Validate the pre-processing
def validate_data(data):
    
    tkList = data["tokenList"]
    raws = data["raws"]
    
    for (tokens, raw) in zip(tkList,raws):
#         print(tokens)
        
#         print(raw)
        temp  = "".join(tokens)
#         print(temp)
        assert(temp == raw)
        if not (temp == raw):
            print(temp)
            print(raw)
            print(tokens)
validate_data(train)

In [9]:
train.head(5)

,input,raws,tokenList
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...,迈向充满希望的新世纪——一九九八年新年讲话（附图片１张）,"[迈向, 充满, 希望, 的, 新, 世纪, ——, 一九九八年, 新年, 讲话, （, 附..."
1,中共中央 总书记 、 国家 主席 江 泽民,中共中央总书记、国家主席江泽民,"[中共中央, 总书记, 、, 国家, 主席, 江, 泽民]"
2,（ 一九九七年 十二月 三十一日 ）,（一九九七年十二月三十一日）,"[（, 一九九七年, 十二月, 三十一日, ）]"
3,１２月 ３１日 ， 中共中央 总书记 、 国家 主席 江 泽民 发表 ...,１２月３１日，中共中央总书记、国家主席江泽民发表１９９８年新年讲话《迈向充满希望的新世纪》。...,"[１２月, ３１日, ，, 中共中央, 总书记, 、, 国家, 主席, 江, 泽民, 发表,..."
4,同胞 们 、 朋友 们 、 女士 们 、 先生 们 ：,同胞们、朋友们、女士们、先生们：,"[同胞, 们, 、, 朋友, 们, 、, 女士, 们, 、, 先生, 们, ：]"


In [10]:
print(len(train_dic_char))
print(len(set(train_dic_char)))
print(len(train_dic_word))
print(len(set(train_dic_word)))


4698
4698
55303
55303


In [11]:
train_words = []
with open("./corpus/cws/icwb2-data/gold/pku_training_words.utf8", 'r') as f:
    content = f.readlines()  
    print(len(content))
    

55303


In [66]:
# Feature extraction

oov = len(train_dic_char)
print(oov)
def token_index(tok):
    ind = tok
    if tok in train_dic_char:  # if token in vocabulary
        ind = train_dic_char.index(tok)
    else:  # else it's OOV
        ind = oov
    return ind

def str_token_index(string):
    return [token_index(x)   for x in string]




# training labels: convert BIO to integers
# def bio_index(bio):
#     ind = bio
#     if not pd.isnull(bio):  # deal with empty lines
#         if bio=='B':
#             ind = 0
#         elif bio=='I':
#             ind = 1
#         elif bio=='O':
#             ind = 2
#     return ind

# Get Begin Middle End Single sequence
# B 0 M 1 E 2 S 3

def str_bmes_idx(tokenList):
    answer = []
    for item in tokenList:
        if len(item) == 0:
            raise NameErro("Zero Length Word")
        if len(item) == 1:
            answer.append(3)
        else:
            answer.append(0)
            for item in range(len(item) - 2):
                answer.append(1)
            answer.append(2)
    return answer




def extract_features(data_set, istest=False):
    data_temp = data_set.copy()
    

    # Idx for chars
    with Pool(8) as p:
        tokinds = p.map(str_token_index,data_temp['raws'])
#     tokinds = [list(map(token_index, u)) for u in data_temp['raws']]
    data_temp["tokenIdx"] = tokinds
    
    # BIO
    data_temp["bmes"] = [str_bmes_idx(u) for u in data_temp['tokenList']]
#     print(data_temp["bmes"])
    assert (list(map(len,data_temp["bmes"])) == list(map(len,data_temp["tokenIdx"])))
#    assert reduce(operator.and_, list(map(len,data_temp["bmes"])) == list(map(len,data_temp["tokenIdx"])))
    
    
    
    
    
    
    return data_temp
#     txt['token_indices'] = tokinds
#     if not istest:  # can't do this with the test set
#         bioints = [bio_index(b) for b in txt['bio_only']]
#         txt['bio_only'] = bioints
#     return txt

4698


In [67]:
%%time
# MultiThreading
temp = extract_features(train)
tempa = temp

True
:D
CPU times: user 423 ms, sys: 163 ms, total: 586 ms
Wall time: 14.1 s


In [34]:
%%time
temp = extract_features(train)
tempb = temp

CPU times: user 1min 36s, sys: 27.9 ms, total: 1min 36s
Wall time: 1min 36s


In [38]:
assert reduce(operator.and_, tempa["tokenIdx"] == tempb["tokenIdx"])

In [68]:
temp.head(5)

,input,raws,tokenList,tokenIdx,bmes
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...,迈向充满希望的新世纪——一九九八年新年讲话（附图片１张）,"[迈向, 充满, 希望, 的, 新, 世纪, ——, 一九九八年, 新年, 讲话, （, 附...","[84, 2326, 46, 166, 610, 719, 2511, 4631, 134,...","[0, 2, 0, 2, 0, 2, 3, 3, 0, 2, 0, 2, 0, 1, 1, ..."
1,中共中央 总书记 、 国家 主席 江 泽民,中共中央总书记、国家主席江泽民,"[中共中央, 总书记, 、, 国家, 主席, 江, 泽民]","[3347, 3041, 3347, 2471, 748, 1363, 1566, 3411...","[0, 1, 1, 2, 0, 1, 2, 3, 0, 2, 0, 2, 3, 0, 2]"
2,（ 一九九七年 十二月 三十一日 ）,（一九九七年十二月三十一日）,"[（, 一九九七年, 十二月, 三十一日, ）]","[1978, 2468, 226, 226, 1833, 4457, 2831, 2242,...","[3, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 1, 2, 3]"
3,１２月 ３１日 ， 中共中央 总书记 、 国家 主席 江 泽民 发表 ...,１２月３１日，中共中央总书记、国家主席江泽民发表１９９８年新年讲话《迈向充满希望的新世纪》。...,"[１２月, ３１日, ，, 中共中央, 总书记, 、, 国家, 主席, 江, 泽民, 发表,...","[1954, 330, 1504, 807, 1954, 3037, 1902, 3347,...","[0, 1, 2, 0, 1, 2, 3, 0, 1, 1, 2, 0, 1, 2, 3, ..."
4,同胞 们 、 朋友 们 、 女士 们 、 先生 们 ：,同胞们、朋友们、女士们、先生们：,"[同胞, 们, 、, 朋友, 们, 、, 女士, 们, 、, 先生, 们, ：]","[32, 319, 3474, 3411, 2463, 3291, 3474, 3411, ...","[0, 2, 3, 3, 0, 2, 3, 3, 0, 2, 3, 3, 0, 2, 3, 3]"


In [ ]:
def find_longest_sequence(txt):
  '''find the longest sequence in the dataframe'''
  for i in txt.index:
    seqlen = len(txt['token'][i])
    if seqlen > longest_seq:  # update high water mark if new longest sequence encountered
      longest_seq = seqlen
  return longest_seq

train_longest = find_longest_sequence(train_seqs,0)
print('The longest sequence in the training set is %i tokens long' % train_longest)